<a href="https://colab.research.google.com/github/DaAnMaGi/PI_ML_OPS/blob/main/formulacion/formulacion_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descarga de bibliotecas a utilizar

In [4]:
# Se descargan las bibliotecas necesarias
!pip install nltk
# !pip install googletrans
!pip install unidecode
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=f60bf9681295429f0cb6d05f470ce466a8dc455e4a7a1c80032a025ee25f6743
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [134]:
# Se importan las bibliotecas a utilizar

# Pandas y numpy
import pandas as pd
import numpy as np

# Trabajo de archivos
import gzip
import json
from pandas import json_normalize

# Detección de idioma
from googletrans import Translator
translator = Translator()
from langdetect import detect, LangDetectException

#
import itertools

# Gráficación
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Análisis de lenguaje natural
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
from nltk.corpus import wordnet
import nltk
import re

In [128]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [129]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [130]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Se cargan las bases

In [20]:
# Se traen las rutas de los archivos a trabajar.
games_ruta = "/content/games_clean.json.gz"
genres_ruta = "/content/genres_games.json.gz"
specs_ruta = "/content/specs_games.json.gz"
reviews_ruta = "/content/reviews_limpia.json.gz"
items_ruta = "/content/items_limpia.json.gz"

In [137]:
# Se convierten a pandas.

In [8]:
games = pd.read_json(games_ruta,compression="gzip")
games.head()

,name,release_date,id_game,developer
0,Lost Summoner Kitty,2018-01-04,761140,Kotoshiro
1,Ironbound,2018-01-04,643980,Secret Level SRL
2,Real Pool 3D - Poolians,2017-07-24,670290,Poolians.com
3,弹炸人2222,2017-12-07,767400,彼岸领域
4,Log Challenge,None,773570,None


In [9]:
genres = pd.read_json(genres_ruta,compression="gzip")
genres.head()

,id_game,genre
0,761140,Strategy
1,761140,Action
2,761140,Indie
3,761140,Casual
4,761140,Simulation


In [10]:
specs = pd.read_json(specs_ruta,compression="gzip")
# specs["id_game"] = specs["id_game"].astype(int)
specs.head()

,id_game,specs
0,761140,Single-player
1,643980,Single-player
2,643980,Multi-player
3,643980,Online Multi-Player
4,643980,Cross-Platform Multiplayer


In [41]:
reviews = pd.read_json(reviews_ruta,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
reviews.head()

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08


In [21]:
items = pd.read_json(items_ruta,compression="gzip")
items

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...
5159890,76561198329548331,346330,BrainBread 2,0,0
5159891,76561198329548331,373330,All Is Dust,0,0
5159892,76561198329548331,388490,One Way To Die: Steam Edition,3,3
5159893,76561198329548331,521570,You Have 10 Seconds 2,4,4


# Procesamiento del lenguaje natural en las reviews:

1.   Identificación de idioma de cada review.
2.   Separación de las reviews en inglés.
3.   Procesamiento de las reviews para obtenerlas lematizadas.



In [42]:
reviews

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08
...,...,...,...,...,...
43971,wayfeng,730,True,its FUNNNNNNNN,2015-10-14
43972,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10
43973,72947282842,730,True,Prettyy Mad Game,2015-10-31
43974,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14


In [43]:
# Se obtiene información sobre la distribución del dataset.
reviews["recommend"].value_counts(normalize=True)

True     0.906972
False    0.093028
Name: recommend, dtype: float64

In [44]:
# Se transliteran los caracteres de "review" a ASCII
reviews["review"] = reviews["review"].apply(unidecode)
# Se colocan todos los caracteres en minúscula y se asegura que sean strings.
reviews["review"] = reviews["review"].str.lower()

In [45]:
# Se crea una función para detectar el lenguaje (ya que aplicar limpieza de manera directa no estaba dando resultado)
# En caso de que no sea capaz de detectarlo, no trae ningún valor.
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException as e:
        return None

In [46]:
# Se detecta el lenguaje para cada review
reviews["language"] = reviews["review"].apply(detect_language)

In [47]:
# Se revisa la base obtenida.
reviews

,user_id,item_id,recommend,review,review_date,language
0,76561197970982479,1250,True,simple yet with great replayability. in my opi...,2011-11-05,en
1,76561197970982479,22200,True,it's unique and worth a playthrough.,2011-07-15,en
2,76561197970982479,43110,True,great atmosphere. the gunplay can be a bit chu...,2011-04-21,en
3,js41637,251610,True,i know what you think when you see this title ...,2014-06-24,en
4,js41637,227300,True,for a simple (it's actually not all that simpl...,2013-09-08,en
...,...,...,...,...,...,...
43971,wayfeng,730,True,its funnnnnnnn,2015-10-14,no
43972,76561198251004808,253980,True,awesome fantasy game if you don't mind the gra...,2015-10-10,en
43973,72947282842,730,True,prettyy mad game,2015-10-31,no
43974,ApxLGhost,730,True,amazing game 10/10,2015-12-14,tl


In [50]:
# Se decide guardar la base con los idiomas detectados.

def guardar_archivo(DataFrame,RutaNuevoArchivo):
  ''' Convierte un Dataframe dado en json y lo comprime en formato gzip'''
  archivo_json = DataFrame.to_json()
  with gzip.open(RutaNuevoArchivo, "w") as f:
    f.write(archivo_json.encode('utf-8'))

guardar_archivo(reviews,"/content/reviews_lang.json.gz")


In [49]:
# Se decide seguir trabajando con la nueva base de reviews a partir del nuevo archivo
ruta_reviews_lang = "/content/reviews_lang.json.gz"
reviews = pd.read_json(ruta_reviews_lang,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
reviews

,user_id,item_id,recommend,review,review_date,language
0,76561197970982479,1250,True,simple yet with great replayability. in my opi...,2011-11-05,en
1,76561197970982479,22200,True,it's unique and worth a playthrough.,2011-07-15,en
2,76561197970982479,43110,True,great atmosphere. the gunplay can be a bit chu...,2011-04-21,en
3,js41637,251610,True,i know what you think when you see this title ...,2014-06-24,en
4,js41637,227300,True,for a simple (it's actually not all that simpl...,2013-09-08,en
...,...,...,...,...,...,...
43971,wayfeng,730,True,its funnnnnnnn,2015-10-14,no
43972,76561198251004808,253980,True,awesome fantasy game if you don't mind the gra...,2015-10-10,en
43973,72947282842,730,True,prettyy mad game,2015-10-31,no
43974,ApxLGhost,730,True,amazing game 10/10,2015-12-14,tl


In [51]:
# Se revisan los idiomas obtenidos a partir de la detección.
reviews["language"].unique()

# Es importante notar que hay reviews (por ejemplo, del inglés y del español)
# que fueron detectadas en otros idiomas, esto debido a errores de escritura
# u ortografía.

array(['en', 'tl', 'so', 'sv', 'af', 'cy', 'pl', 'et', 'pt', 'sl', 'fi',
       'hr', 'id', 'lt', 'no', 'hu', 'da', 'it', 'sw', 'nl', 'fr', 'ro',
       'es', None, 'cs', 'ca', 'de', 'tr', 'sk', 'sq', 'lv', 'vi'],
      dtype=object)

In [52]:
# Se obtiene la información del dataframe por lenguaje.
reviews.groupby(["language"],dropna=False).count()

,user_id,item_id,recommend,review,review_date
language,,,,,
af,837,837,837,837,837
ca,271,271,271,271,271
cs,43,43,43,43,43
cy,509,509,509,509,509
da,635,635,635,635,635
de,111,111,111,111,111
en,32983,32983,32983,32983,32983
es,1059,1059,1059,1059,1059
et,371,371,371,371,371


In [59]:
# Se escogen las reviews detectadas correctamente del idioma inglés ("en")
reviews_en = reviews[reviews["language"] == "en"]
reviews_en

,user_id,item_id,recommend,review,review_date,language
0,76561197970982479,1250,True,simple yet with great replayability. in my opi...,2011-11-05,en
1,76561197970982479,22200,True,it's unique and worth a playthrough.,2011-07-15,en
2,76561197970982479,43110,True,great atmosphere. the gunplay can be a bit chu...,2011-04-21,en
3,js41637,251610,True,i know what you think when you see this title ...,2014-06-24,en
4,js41637,227300,True,for a simple (it's actually not all that simpl...,2013-09-08,en
...,...,...,...,...,...,...
43967,76561198236893796,209000,True,"this game is great,this game is very unusual,t...",2015-10-21,en
43968,pigeonie,440,True,"a pretty fun game, i got addicted pretty quick.",2015-09-04,en
43969,76561198237650078,730,True,the only game i need.,2015-11-28,en
43972,76561198251004808,253980,True,awesome fantasy game if you don't mind the gra...,2015-10-10,en


In [60]:
# Se obtiene información sobre la distribución del dataset.
reviews_en["recommend"].value_counts(normalize=True)

True     0.899797
False    0.100203
Name: recommend, dtype: float64

In [145]:
# Se crea la lista de stopwords, como estamos trabajando con los reviews en inglés,
# trabajaremos con estos.
stopwords = nltk.corpus.stopwords.words('english')
stopwords = list(stopwords)

In [154]:
# Se agregan a la lista de stopwords, después de una primera iteración palabras
# que después no representan información útil.
filtrar = []

if True:
  filtrar.append("game")
  filtrar.append("games")

In [141]:
# se crea la siguiente función para obtener la categoría de la palabra que se usará
# para la lematización
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


In [155]:
wnl.lemmatize("boobs", get_wordnet_pos("boobs"))

'boob'

In [160]:
# Se limpia, tokeniza y lematiza cada una de las
# reviews de la base de datos.

reviews_lemma = []

for review in reviews_en.review:
  # Se limpia de caracteres no alfabéticos
  review = re.sub("[^a-zA-Z]"," ",str(review))
  # Se tokeniza.
  review = nltk.word_tokenize(review)

  # Se lematizan las palabras
  review = [wnl.lemmatize(w, get_wordnet_pos(w)) for w in review]

  # Se eliminan palabras menores de 3 letras
  review = [word for word in review if len(word)>3]
  # Sacamos las Stopwords
  review = [word for word in review if not word in stopwords]
  review = [word for word in review if word not in filtrar]

  # Se unifica nuevamente la review
  review = " ".join(review)

  # Se agrega a la lista que luego servirá para agregar la info al dataframe
  reviews_lemma.append(review)


In [167]:
# Se visualiza el proceso.
reviews_lemma[0:40]

['simple great replayability opinion zombie horde team work well left dead plus global leveling system alot earth zombie splatter whole family amaze sort rare',
 'unique worth playthrough',
 'great atmosphere gunplay chunky time definitely worth hope sequel sequel',
 'know think title barbie dreamhouse party intimidate title easily gotys cliche mechanic late simply good core gameplay noscope friend show dance move shame show true fashion color combination know easily range blast play',
 'simple actually simple truck drive simulator quite relax play simple easy basic wasd drive want much harder realistic manually change gear much harder turn reverse imagine would actual truck luckily reverse park extra point cause bloody hard suprisingly nice truck drive',
 'little play bore time passer recommend',
 'elegant integration gameplay story world development aesthetic',
 'random drop random quest stat point animation style reminiscent voodoo card',
 'balance tactic strategy potential reward b

In [162]:
# Se agrega la lematización a el dataset.
reviews_en["review_lemma"] = reviews_lemma

<ipython-input-162-a07eb0bec4c8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_en["review_lemma"] = reviews_lemma


In [168]:
# Se visualiza el dataset
reviews_en

,user_id,item_id,recommend,review,review_date,language,review_lemma
0,76561197970982479,1250,True,simple yet with great replayability. in my opi...,2011-11-05,en,simple great replayability opinion zombie hord...
1,76561197970982479,22200,True,it's unique and worth a playthrough.,2011-07-15,en,unique worth playthrough
2,76561197970982479,43110,True,great atmosphere. the gunplay can be a bit chu...,2011-04-21,en,great atmosphere gunplay chunky time definitel...
3,js41637,251610,True,i know what you think when you see this title ...,2014-06-24,en,know think title barbie dreamhouse party intim...
4,js41637,227300,True,for a simple (it's actually not all that simpl...,2013-09-08,en,simple actually simple truck drive simulator q...
...,...,...,...,...,...,...,...
43967,76561198236893796,209000,True,"this game is great,this game is very unusual,t...",2015-10-21,en,great unusual amaze batman
43968,pigeonie,440,True,"a pretty fun game, i got addicted pretty quick.",2015-09-04,en,pretty addict pretty quick
43969,76561198237650078,730,True,the only game i need.,2015-11-28,en,need
43972,76561198251004808,253980,True,awesome fantasy game if you don't mind the gra...,2015-10-10,en,awesome fantasy mind graphic recommend


In [164]:
# Se procede a guardar la base con los datos lematizados para futuras iteraciones:
guardar_archivo(reviews_en,"/content/reviews_lemma.json.gz")

# Análisis de las reviews lematizadas.

In [166]:
ruta_reviews_lemma = "/content/reviews_lemma.json.gz"
reviews = pd.read_json(ruta_reviews_lemma,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
reviews

,user_id,item_id,recommend,review,review_date,language,review_lemma
0,76561197970982479,1250,True,simple yet with great replayability. in my opi...,2011-11-05,en,simple great replayability opinion zombie hord...
1,76561197970982479,22200,True,it's unique and worth a playthrough.,2011-07-15,en,unique worth playthrough
2,76561197970982479,43110,True,great atmosphere. the gunplay can be a bit chu...,2011-04-21,en,great atmosphere gunplay chunky time definitel...
3,js41637,251610,True,i know what you think when you see this title ...,2014-06-24,en,know think title barbie dreamhouse party intim...
4,js41637,227300,True,for a simple (it's actually not all that simpl...,2013-09-08,en,simple actually simple truck drive simulator q...
...,...,...,...,...,...,...,...
43967,76561198236893796,209000,True,"this game is great,this game is very unusual,t...",2015-10-21,en,great unusual amaze batman
43968,pigeonie,440,True,"a pretty fun game, i got addicted pretty quick.",2015-09-04,en,pretty addict pretty quick
43969,76561198237650078,730,True,the only game i need.,2015-11-28,en,need
43972,76561198251004808,253980,True,awesome fantasy game if you don't mind the gra...,2015-10-10,en,awesome fantasy mind graphic recommend
